# Steam Template Matching
Reference: https://github.com/woctezuma/steam-template-matching

## Install Python packages

In [ ]:
%cd /content
%pip install -q gamedatacrunch img2dataset mediapy

## Download images from Steam (only the first time)

In [ ]:
def get_image_url(app_id):
  return f'https://cdn.cloudflare.steamstatic.com/steam/apps/{app_id}/capsule_231x87.jpg'

def write_to_text_file(app_ids, fname):
  with open(fname, 'w') as f:
    for app_id in app_ids:
      url = get_image_url(app_id)
      f.write(f'{url}\n')

In [ ]:
import gamedatacrunch as gdc

app_ids = gdc.load_app_ids()
write_to_text_file(app_ids, fname='myimglist.txt')

In [ ]:
!echo img2dataset --url_list=myimglist.txt --output_folder=steam_images --resize_mode=no
!echo zip -r steam_images.zip steam_images

## Download images from Github (once archived the first time)

In [ ]:
!curl -O -L https://github.com/woctezuma/steam-template-matching/releases/download/capsule_231x87/steam_images.zip
!unzip -q steam_images.zip

## Download the template

In [ ]:
!curl -O -L https://github.com/woctezuma/steam-template-matching/releases/download/capsule_231x87/template.png

In [ ]:
import mediapy as media
import skimage.io

from skimage.transform import rescale

def load_template_from_disk(scale=1.0, verbose=True):
  template_fname = 'template.png'
  template = skimage.io.imread(template_fname)

  # Remove the transparency channel
  template = template[..., :3]
  # *Caveat*: this is important, because the template
  # is a PNG file while the other images are JPG files!

  if scale != 1.0:
    template = rescale(template, (scale, scale, 1))

  if verbose:
    media.show_image(template)
    print(template.shape)

  return template

template = load_template_from_disk()

## Template Matching

In [ ]:
import glob

def get_test_fnames(image_folder, file_ext = '.jpg'):
  return sorted(glob.glob(image_folder +'/*' + file_ext))

test_fnames = get_test_fnames('steam_images/*')
print(f'#images = {len(test_fnames)}')

### Example

In [ ]:
import numpy as np
from skimage.feature import match_template

fname = "steam_images/00003/000032503.jpg"
image = skimage.io.imread(fname)

media.show_image(image)
print(image.shape)

result = np.squeeze(match_template(image, template))
score = np.max(result)

In [ ]:
import matplotlib.pyplot as plt

coin = template

# Reference: https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_template.html

ij = np.unravel_index(np.argmax(result), result.shape)
x, y = ij[::-1]

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2)
ax3 = plt.subplot(1, 3, 3, sharex=ax2, sharey=ax2)

ax1.imshow(coin, cmap=plt.cm.gray)
ax1.set_axis_off()
ax1.set_title('template')

ax2.imshow(image, cmap=plt.cm.gray)
ax2.set_axis_off()
ax2.set_title('image')
# highlight matched region
hcoin, wcoin, num_channels = coin.shape
rect = plt.Rectangle((x, y), wcoin, hcoin, edgecolor='r', facecolor='none')
ax2.add_patch(rect)

ax3.imshow(np.squeeze(result))
ax3.set_axis_off()
ax3.set_title('`match_template`\nresult')
# highlight matched region
ax3.autoscale(False)
ax3.plot(x, y, 'o', markeredgecolor='r', markerfacecolor='none', markersize=10)

plt.show()

### Run

In [ ]:
template_scale = 1.0
template = load_template_from_disk(template_scale)

In [ ]:
start_fname = ""

if start_fname is None or len(start_fname) == 0:
  start_index = 0
else:
  start_index = test_fnames.index(start_fname)

if start_index == 0:
  d = {}
  max_score = 0
  best_fname = None

for counter, fname in enumerate(test_fnames[start_index:], start=1):
  if counter % 1000 == 0:
    print(f'Currently at {fname}. So far, max = {max_score:.2} at {best_fname}')

  image = skimage.io.imread(fname)
  
  try:
    result = np.squeeze(match_template(image, template))
  except ValueError:
    print(f'Skipping {fname}')
    continue

  score = np.max(result)  
  d[fname] = score

  if score > max_score:
    max_score = score
    best_fname = fname

    media.show_image(image)
    print(f"{fname} score: {score}")

print(f'In the end, max = {max_score:.2} at {best_fname}')